In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from repltilian import SwiftREPL
repl = SwiftREPL()

print("REPL is running !")
REPL is running !
 11>  


In [2]:
repl.add_reload_file("demo.swift")

In [3]:
repl.run("""
var point = Point<Float>(x: 1, y: 2)
""", reload=True);

 46.  
 47. var point = Point<Float>(x: 1, y: 2) 
 48.  
point: Point<Float> = {
  x = 1
  y = 2
}
 48>  


In [4]:
repl.run("""
var point1 = Point<Float>(x: 5, y: 2)
var point2 = Point<Float>(x: 5, y: 2)
let result = point1 + point2 + point
result.translate(dx: 1, dy: 100)
""");

 48>  
 49. var point1 = Point<Float>(x: 5, y: 2) 
 50. var point2 = Point<Float>(x: 5, y: 2) 
 51. let result = point1 + point2 + point 
 52. result.translate(dx: 1, dy: 100) 
 53.  
$R0: Point<Float> = {
  x = 12
  y = 106
}
point1: Point<Float> = {
  x = 5
  y = 2
}
point2: Point<Float> = {
  x = 5
  y = 2
}
result: Point<Float> = {
  x = 11
  y = 6
}
 53>  


In [5]:
repl.vars["point2"].json()

{'x': 5, 'y': 2}

In [7]:
repl.vars["result"].json()

{'x': 11, 'y': 6}

In [8]:
repl.vars["result"] = "Point<Float>", {'x': -1, 'y': -1}

In [9]:
repl.vars["result"].json()

{'x': -1, 'y': -1}

In [10]:
repl.run("result.translate(dx: 1, dy: 100)");

 59> result.translate(dx: 1, dy: 100)
$R5: Point<Float> = {
  x = 0
  y = 99
}
 60>  
